In [108]:
# coding=utf-8
import MySQLdb
import codecs
from datetime import *
import time
from FundManager import *

import sys
sys.path.append('../')
from DB import *

class SalaryFund:
    def __init__(self):
        self.db = DB()
    
    # 获取最新的一条记录的日期
    def getLastDate(self):
        result = self.db.executeSql("SELECT trade_date FROM salary_fund ORDER BY trade_date DESC LIMIT 1")
        return result[0][0]
    
    # 根据fundbar数据确定给定的date是否是交易日
    def isTradeDate(self, date):
        sql = "SELECT trade_date FROM fund_bar WHERE trade_date = '%s'" % date.strftime("%Y-%m-%d")
        result = self.db.executeSql(sql)
        return len(result) > 0
        
    # 获取指定日期的fund信息, 如果是周一，则上一个交易日的
    def getFundByDate(self, date):
        sql = "SELECT trade_date,total_assets,security_assets,cash,share,net_value FROM salary_fund WHERE trade_date <= '%s' ORDER BY trade_date DESC LIMIT 1"
        sql = sql % date.strftime("%Y-%m-%d")
        result = self.db.executeSql(sql)
        if len(result) == 0:
            raise Exception("Failed to get salary fund equal or befor date : %s" % date.strftime("%Y-%m-%d"))
        else:
            result = result[0]
            data = {"trade_date":result[0], "total_assets":result[1], "security_assets":result[2], \
                    "cash":result[3], "share":result[4], "net_value":result[5]}
            return data
    
    # 获取指定日期的交易
    def getFundChangesByDate(self, date):
        sql = "SELECT trade_date, trade_type, code, money, share, share_net_value, total_money FROM salary_fund_changes WHERE trade_date = '%s';"
        sql = sql % date.strftime("%Y-%m-%d")
        result = self.db.executeSql(sql)
        if len(result) == 0:
            return []
        else:
            data = []
            for item in result:
                data.append({"trade_date":item[0], "trade_type":item[1], "code":item[2], \
                             "money":item[3], "share":item[4], "share_net_value":item[5],"total_money":item[6]})
            return data
        
    def getFundValueByDate(self, code, date):
        datestr = date.strftime("%Y-%m-%d")
        sql = "SELECT net_asset_value from fund_bar where code = '%s' and trade_date = '%s';" % (code, datestr)
        result = self.db.executeSql(sql)
        if len(result) > 0:
            return result[0][0]
        else:
            raise Exception("Failed to get fund value, code = %s, date = %s" % (code, datestr))
        
    # 计算账户中的证券部分的价值
    # 将账户中下雨等于date的所有操作都计算进来，计算每一个基金的份额，乘以当前的基金净值，得到总的证券价值
    # 正常当天的价值需要在第二天才能计算，因为ETF等基金净值通常要晚上8，9点之后才会更新
    # 计算证券价值只需考虑买入和卖出基金的操作，现金注入和取出不用考虑
    def calcSecurityValue(self, date):
        datestr = date.strftime("%Y-%m-%d")
        sql = "SELECT code,share,trade_type from salary_fund_changes where trade_date <= '%s' order by trade_date" % datestr
        result = self.db.executeSql(sql)
        ## (('512580', 5200.0, 'buy'), ('512000', 5800.0, 'sell'))
        total_value = 0
        for item in result:
            if item[2] == "buy":
                net_value = self.getFundValueByDate(item[0], date)
                total_value += item[1] * net_value
            elif item[2] == "sell":
                net_value = self.getFundValueByDate(item[0], date)
                total_value -= item[1] * net_value
        return total_value
        
    # 购买基金，注入现金，取出现金会导致份额变化
    # 在前一天的净值和份额的基础上，加上当天的操作带来的这些份额变化
    def calcShares(self, date):
        prev_date = date - timedelta(days=1)
        prev_fund = self.getFundByDate(prev_date)
        prev_net_value = prev_fund['net_value']
        prev_share = prev_fund['share']
        share = prev_share
        changes = self.getFundChangesByDate(date)
        for change in changes:
            if change["trade_type"] == "cashout" and change['total_money'] > 0:
                change['total_money'] = -change['total_money']
            if change["trade_type"] in ["buy", "cashin", "cashout"]:
                share += change['total_money'] / prev_net_value
        return share
    
    # 计算指定日期的cash剩余
    # 基金卖出，注入现金，取出现金会导致现金变化
    def calcCash(self, date):
        prev_date = date - timedelta(days=1)
        prev_fund = self.getFundByDate(prev_date)
        prev_cash = prev_fund['cash']
        cash = prev_cash
        changes = self.getFundChangesByDate(date)
        for change in changes:
            if change["trade_type"] == "cashout" and change['total_money'] > 0:
                change['total_money'] = -change['total_money']
            if change["trade_type"] in ["sell", "cashin", "cashout"]:
                cash += change['total_money']
        return cash
    
    def saveFund(self, fund):
        sql = "INSERT INTO salary_fund (trade_date, total_assets, security_assets, cash, share, net_value) VALUES ('%s', %f, %f, %f, %f, %f)"
        sql = sql % (fund['trade_date'], fund['total_assets'], fund['security_assets'], fund['cash'], fund['share'], fund['net_value'])
        self.db.executeSql(sql)
        
        
    # 计算指定日期的基金份额
    # 在前一天的份额和净值的基础上，分三种情况更新份额：
    # - day2 买入一只基金，总共花去4000，手续费5元，当天的操作都不会计算到前一天净值里，等明天算今天的净值才要考虑
    # - day3 先按照day1净值买入4000基金，计算获得多少份额
    # - day3 计算day2的证券部分的价值
    # - day3 (证券价值+现金)/总份额 = day2净值
    # 
    # - day6 取出现金x
    # - day7 按照day5的净值计算取出现金x的份额，减去该份额，减去现金x
    # - day7 计算day6的证券部分的价值
    # - day7 (证券价值+现金)/总份额 = day6净值
    # 
    # * day8 注入现金y
    # * day9 按照day7的净值计算现金y的份额，加上份额，加上现金y
    # * day9 计算day8的证券部分的价值
    # * day9 (证券价值+现金)/总份额 = day8净值
    def updateFundByDate(self, date):
        if not self.isTradeDate(date):
            raise Exception("Date %s is a invalid trade date" % date.strftime("%Y-%m-%d"))
        share = self.calcShares(date)
        security_assets = self.calcSecurityValue(date)
        cash = self.calcCash(date)
        total_money = security_assets + cash
        net_value = total_money / share 
        
        fund = {"trade_date": date.strftime("%Y-%m-%d"), "total_assets": total_money, "security_assets": security_assets,\
               "cash": cash, "share": share, "net_value": net_value}
        self.saveFund(fund)
        print(fund)
    
    # 更新salary fund，计算所有未计算的数据并插入数据库
    def updateAllFund(self):
        date = self.getLastDate() + timedelta(days=1)
        end = datetime.now().date()
        print("From", date, "to", end)
        while date < end:
            if fund.isTradeDate(date):
                fund.updateFundByDate(date)
            date = date + timedelta(days=1)
            
fund = SalaryFund()
fund.updateAllFund()

# date = datetime(2018,4,20)
# while date < datetime(2018,4,21):
#     if fund.isTradeDate(date):
#         fund.calcFundByDate(date)
#     date = date + timedelta(days=1)
    
# fund.updateNetValue(datetime(2018,3,11))
# fund.isTradeDate(datetime(2018,3,17))

From 2018-04-26 to 2018-05-01
{'trade_date': '2018-04-26', 'total_assets': 29000.515472, 'security_assets': 29000.515472, 'cash': 0.0, 'share': 30714.6694728504, 'net_value': 0.9441910321592231}
{'trade_date': '2018-04-27', 'total_assets': 29143.062876999997, 'security_assets': 29143.062876999997, 'cash': 0.0, 'share': 30714.669473, 'net_value': 0.9488320524698617}
